In [ ]:
pip install camel-tools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 35.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 38.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.3 MB/s eta 0:00:00
  Created wheel for camel-kenlm: filename=camel_kenlm-2023.3.17.2-cp310-cp310-linux_x86_64.whl size=3537000 sha256=763dfb

In [ ]:
pip install opencage

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import csv
import opencage.geocoder
import folium
from camel_tools.ner import NERecognizer
from camel_tools.tokenizers.word import simple_word_tokenize
import csv
from collections import Counter
import re
from folium.plugins import PolyLineTextPath
import pandas as pd
import numpy as np

In [ ]:
def write_tuples_to_csv(tuples_list, filename):
    with open(filename, 'w', newline='') as file:
        file.truncate()  # Clear the contents of the file
        writer = csv.writer(file)
        for tup in tuples_list:
            writer.writerow(tup)


In [ ]:
def plot_arabic_places(place_names, api_key):
    # Initialize OpenCage client
    geocoder = opencage.geocoder.OpenCageGeocode(api_key)

    # Geocode place names using OpenCage API
    coords = []
    for tpl in place_names:
        name = "{}, {}".format(tpl[1], tpl[0]) # combine first and second elements of tuple as place name
        result = geocoder.geocode(name)
        if len(result) > 0:
            location = result[0]['geometry']
            coords.append((location['lat'], location['lng'], name))

    # Plot coordinates on a map using Folium
    map = folium.Map(location=[coords[0][0], coords[0][1]], zoom_start=10)
    for coord in coords:
        folium.Marker(location=[coord[0], coord[1]], popup=coord[2], tooltip=coord[2]).add_to(map)
    return map

In [ ]:
def remove_waw_at_beginning(tuples_list):
    # Loop over each tuple in the list
    for i, tpl in enumerate(tuples_list):
        # Check if the first character of the first element is 'و'
        if tpl[0].startswith('و'):
            # Remove the first occurrence of 'و'
            tuples_list[i] = (tpl[0].replace('و', '', 1), *tpl[1:])

    return tuples_list

In [ ]:
def count_locations(input_file, location_column_char):
    # Convert the column character to an integer index
    location_column_index = ord(location_column_char.lower()) - 97

    # Create a counter to keep track of the frequency of each location
    location_counter = Counter()

    # Open the CSV file for reading
    with open(input_file, 'r') as csv_file:
        reader = csv.reader(csv_file)

        # Loop over each row in the CSV file
        for row in reader:
            # Increment the counter for this location
            location_counter[row[location_column_index]] += 1

    # Sort the locations by frequency
    sorted_locations = sorted(location_counter.items(), key=lambda x: x[1], reverse=True)

    # Write the results to a new CSV file
    with open('/content/drive/MyDrive/Dr._Zaki_Assistance/location_frequencies.csv', 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Location', 'Frequency'])
        for location, frequency in sorted_locations:
            writer.writerow([location, frequency])
    return sorted_locations


In [ ]:
def generate_output_csv(input_file_path):
    output_file_path = '/content/drive/MyDrive/Dr._Zaki_Assistance/ARA Project/country_frequency_with_context.csv'
    # Load the input CSV file into a Pandas DataFrame
    df = pd.read_csv(input_file_path)
    # Calculate the frequency of each country
    freqs = df['country'].value_counts()
    # Create a new DataFrame to hold the output
    output_df = pd.DataFrame(columns=['country', 'frequency', 'random_context'])
    # Iterate over each unique country and add a row to the output DataFrame
    for country in freqs.index:
        # Get the frequency of the current country
        freq = freqs[country]
        # Get all contexts for the current country
        contexts = df.loc[df['country'] == country, 'context']
        # Choose a random context from the list
        random_context = np.random.choice(contexts)
        # Add a row to the output DataFrame
        output_df = output_df.append({
            'country': country,
            'frequency': freq,
            'random_context': random_context
        }, ignore_index=True)
    # Save the output DataFrame to a CSV file with the requested name
    output_df.to_csv(output_file_path, index=False)


In [ ]:
def get_context(words, target_word):
    # find the index of the target word
    try:
        index = words.index(target_word)
    except ValueError:
        return None

    # slice the list to get the 10 words before and after the target word
    start_index = max(0, index-30)
    end_index = min(len(words), index+31)
    context_words = words[start_index:end_index]

    # convert the list back to a string and return it
    context = ' '.join(context_words)
    return context


In [ ]:
def remove_english_words(words):
    non_english_words = []
    for word in words:
        if not re.search(r'[a-zA-Z]', word):
            non_english_words.append(word)
    return non_english_words

In [ ]:
def add_column_to_tuples(tuples_list, csv_file):
    # Open the CSV file for reading
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)

        # Create a dictionary to store the values in the CSV file
        values_dict = {}
        for row in reader:
            values_dict[row[0]] = row[1]

    # Loop over each tuple in the list
    updated_tuples_list = []
    for tpl in tuples_list:
        # Check if the second element of the tuple matches a cell in the first column of the CSV file
        if tpl[1] in values_dict:
            # Add the cell in the third column to the tuple
            updated_tuple = tpl + (values_dict[tpl[1]],)
        else:
            updated_tuple = tpl

        updated_tuples_list.append(updated_tuple)

    return updated_tuples_list

In [ ]:
def markers_arabic_places(place_names, api_key):
    # Initialize OpenCage client
    geocoder = opencage.geocoder.OpenCageGeocode(api_key)

    # Geocode place names using OpenCage API
    coords = []
    for tpl in place_names:
        name = "{}, {}".format(tpl[1], tpl[0]) # combine first and second elements of tuple as place name
        result = geocoder.geocode(name)
        if len(result) > 0:
            location = result[0]['geometry']
            coords.append((location['lat'], location['lng'], name))

    # Create list of marker locations
    marker_locations = [(coord[0], coord[1], name) for coord, name in zip(coords, place_names)]

    # Return list of marker locations
    return marker_locations


def draw_arrows_to_main_marker(marker_locations):
    # Initialize the map
    map = folium.Map(location=[35.0, 38.0], zoom_start=6)

    # Add a tile layer to the map (optional)
    folium.TileLayer('openstreetmap').add_to(map)

    # Geocode the location of the main marker
    geocoder = opencage.geocoder.OpenCageGeocode("686e5750dedb47ada75ceca950e88378")
    result = geocoder.geocode("40, سورية", no_annotations=1)
    main_marker_location = (result[0]['geometry']['lat'], result[0]['geometry']['lng'])

    # Create the main marker and add it to the map
    main_marker = folium.Marker(location=main_marker_location)
    main_marker.add_to(map)

    # Create the other markers and add them to the map
    markers = []
    for location in marker_locations:
        # Create a custom HTML string for the popup content with a width of 300px
        popup_html = '<div style="width: 300px;">{}</div>'.format(location[2])
        marker = folium.Marker(location=location[:2], popup=popup_html)
        marker.add_to(map)
        markers.append(marker)

    # Create polylines connecting each marker to the main marker
    for marker in markers:
        polyline = folium.PolyLine(
            locations=[(marker.location[0], marker.location[1]), (main_marker.location[0], main_marker.location[1])],
            color='red',
            weight=3,
            opacity=0.7,
            dash_array='5,10',
            tooltip="Arrow pointing to main marker",
            arrowhead_scale=0.2
        )
        # Add an arrow to the polyline
        polyline.add_to(map)
    # Display the map
    return map


In [ ]:
def csv_to_tuples(file_path):
    # Initialize an empty list to store the tuples
    tuples_list = []

    # Open the CSV file
    with open(file_path, 'r') as csv_file:
        # Create a CSV reader object
        csv_reader = csv.reader(csv_file)

        # Skip the header row
        next(csv_reader)

        # Loop through each row of the CSV file
        for row in csv_reader:
            # Extract the values from the desired columns and create a tuple
            tuple_values = (row[0], row[1], row[2])

            # Append the tuple to the list
            tuples_list.append(tuple_values)

    # Return the list of tuples
    return tuples_list

In [ ]:
ner = NERecognizer('CAMeL-Lab/bert-base-arabic-camelbert-mix-ner')
sentence = simple_word_tokenize('إمارة أبوظبي هي إحدى إمارات دولة الإمارات العربية المتحدة السبع')
ner.predict_sentence(sentence)

['O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'O']

In [ ]:
with open('/content/drive/MyDrive/Dr._Zaki_Assistance/ARA Project/النكبات.txt', 'r') as file:
        text = file.read()
sentence = simple_word_tokenize(text)
#remove the english words from the list
sentence = remove_english_words(sentence)
labels = ner.predict_sentence(sentence)

In [ ]:
sentence

[':',
 '1',
 'النكات',
 'خلاصة',
 'تاريخ',
 'سورية',
 'منذ',
 'العهد',
 'الأول',
 'بعد',
 'الطوفان',
 'إلى',
 'عهد',
 'الجمهورية',
 'بلبنان',
 'أمين',
 'الريحاني',
 ':',
 '3',
 'النكبات',
 'خلاصة',
 'تاريخ',
 'سورية',
 'منذ',
 'العهد',
 'الأول',
 'بعد',
 'الطوفان',
 'إلى',
 'عهد',
 'الجمهورية',
 'بلبنان',
 'تأليف',
 'أمين',
 'الريحاني',
 'هنداوي',
 ':',
 '4',
 'النكبات',
 'أمين',
 'الريحاني',
 'الناشر',
 'مؤسسة',
 'هنداوي',
 'المشهرة',
 'برقم',
 '10585970',
 'بتاريخ',
 '٢٦',
 '/',
 '/',
 '١',
 '٢٠١٧',
 'يورك',
 'هاوس',
 '،',
 'شييت',
 'ستريت',
 '،',
 'وندسور',
 '،',
 '،',
 'المملكة',
 'المتحدة',
 'تليفون',
 ':',
 '٨٣٢٥٢٢',
 '١٧٥٣',
 '(',
 '٠',
 ')',
 '٤٤',
 '+',
 'البريد',
 'الإلكتروني',
 ':',
 '@',
 '.',
 'الموقع',
 'الإلكتروني',
 ':',
 ':',
 '/',
 '/',
 '.',
 '.',
 'إن',
 'مؤسسة',
 'هنداوي',
 'غير',
 'مسئولة',
 'عن',
 'آراء',
 'المؤلف',
 'وأفكاره',
 '،',
 'وإنما',
 'يعبر',
 'الكتاب',
 'عن',
 'آراء',
 'مؤلفه',
 '.',
 'تصميم',
 'الغلاف',
 ':',
 'ليلى',
 'يسري',
 'الترقيم',
 'الدولي',
 

In [ ]:
# Print each token paired with it's NER label
ner_lst = list(zip(sentence, labels))
location_filtered_tuples = [tup for tup in ner_lst if 'B-LOC' in tup]

print((location_filtered_tuples))

[('سورية', 'B-LOC'), ('بلبنان', 'B-LOC'), ('سورية', 'B-LOC'), ('بلبنان', 'B-LOC'), ('المملكة', 'B-LOC'), ('غسان', 'B-LOC'), ('بابل', 'B-LOC'), ('أميريكة', 'B-LOC'), ('لبنان', 'B-LOC'), ('سورية', 'B-LOC'), ('الشام', 'B-LOC'), ('سورية', 'B-LOC'), ('سورية', 'B-LOC'), ('الشام', 'B-LOC'), ('شام', 'B-LOC'), ('آشورية', 'B-LOC'), ('صور', 'B-LOC'), ('سورية', 'B-LOC'), ('البحر', 'B-LOC'), ('البحر', 'B-LOC'), ('العقبة', 'B-LOC'), ('الروم', 'B-LOC'), ('مصر', 'B-LOC'), ('طوروس', 'B-LOC'), ('دمشق', 'B-LOC'), ('قرقميش', 'B-LOC'), ('جرابلس', 'B-LOC'), ('دمشق', 'B-LOC'), ('طرطوس', 'B-LOC'), ('صور', 'B-LOC'), ('فلسطين', 'B-LOC'), ('كنعان', 'B-LOC'), ('حام', 'B-LOC'), ('كنعان', 'B-LOC'), ('لبنان', 'B-LOC'), ('وسورية', 'B-LOC'), ('الحثيين', 'B-LOC'), ('الفرات', 'B-LOC'), ('مصر', 'B-LOC'), ('يشوع', 'B-LOC'), ('كنعان', 'B-LOC'), ('يهوذا', 'B-LOC'), ('بني', 'B-LOC'), ('دمشق', 'B-LOC'), ('يهوذا', 'B-LOC'), ('وإسرائيل', 'B-LOC'), ('إسرائيل', 'B-LOC'), ('شكيم', 'B-LOC'), ('نابلس', 'B-LOC'), ('إسرائيل', 'B-LOC')

In [ ]:
#Adding the context of locations to'location_filtered_tuples'
context_lst = []
for i in location_filtered_tuples:
  context_lst.append(get_context(sentence,i[0]))

location_filtered_tuples_with_description = [(t[0], t[1], context_lst[i]) for i, t in enumerate(location_filtered_tuples)]

In [ ]:
location_filtered_tuples_with_description = remove_waw_at_beginning(location_filtered_tuples_with_description)

In [ ]:
location_filtered_tuples_with_description

[('سورية',
  'B-LOC',
  ': 1 النكات خلاصة تاريخ سورية منذ العهد الأول بعد الطوفان إلى عهد الجمهورية بلبنان أمين الريحاني : 3 النكبات خلاصة تاريخ سورية منذ العهد الأول بعد الطوفان إلى عهد الجمهورية بلبنان تأليف أمين الريحاني هنداوي'),
 ('بلبنان',
  'B-LOC',
  ': 1 النكات خلاصة تاريخ سورية منذ العهد الأول بعد الطوفان إلى عهد الجمهورية بلبنان أمين الريحاني : 3 النكبات خلاصة تاريخ سورية منذ العهد الأول بعد الطوفان إلى عهد الجمهورية بلبنان تأليف أمين الريحاني هنداوي : 4 النكبات أمين الريحاني الناشر مؤسسة هنداوي المشهرة'),
 ('سورية',
  'B-LOC',
  ': 1 النكات خلاصة تاريخ سورية منذ العهد الأول بعد الطوفان إلى عهد الجمهورية بلبنان أمين الريحاني : 3 النكبات خلاصة تاريخ سورية منذ العهد الأول بعد الطوفان إلى عهد الجمهورية بلبنان تأليف أمين الريحاني هنداوي'),
 ('بلبنان',
  'B-LOC',
  ': 1 النكات خلاصة تاريخ سورية منذ العهد الأول بعد الطوفان إلى عهد الجمهورية بلبنان أمين الريحاني : 3 النكبات خلاصة تاريخ سورية منذ العهد الأول بعد الطوفان إلى عهد الجمهورية بلبنان تأليف أمين الريحاني هنداوي : 4 النكبات

In [ ]:
#convert tuple to CSV:
csv_output_file = '/content/drive/MyDrive/Dr._Zaki_Assistance/ARA Project/countries - Sheet1.csv'
write_tuples_to_csv(location_filtered_tuples_with_description, csv_output_file)

In [ ]:
#list of Tuples
locations_freq= count_locations("/content/drive/MyDrive/Dr._Zaki_Assistance/ARA Project/countries - Sheet1.csv", 'A')

In [ ]:
locations_freq

[('سورية', 40),
 ('دمشق', 28),
 ('الشام', 27),
 ('مصر', 17),
 ('بابل', 13),
 ('العراق', 10),
 ('غسان', 9),
 ('يهوذا', 9),
 ('آشور', 9),
 ('فلسطين', 8),
 ('الأنباط', 7),
 ('كنعان', 6),
 ('إسرائيل', 6),
 ('اليمن', 6),
 ('اليونان', 6),
 ('رومة', 6),
 ('صور', 5),
 ('البتراء', 5),
 ('الروم', 4),
 ('فينيقية', 4),
 ('تدمر', 4),
 ('فارس', 4),
 ('تنوخ', 4),
 ('بالشام', 4),
 ('لبنان', 3),
 ('البحر', 3),
 ('قرقميش', 3),
 ('أورشليم', 3),
 ('دومة', 3),
 ('أفريقية', 3),
 ('الأردن', 3),
 ('أنطاكية', 3),
 ('حمص', 3),
 ('بدمشق', 3),
 ('القسطنطينية', 3),
 ('أوروبة', 3),
 ('الحجاز', 3),
 ('حلب', 3),
 ('بلبنان', 2),
 ('المملكة', 2),
 ('العقبة', 2),
 ('الحثيين', 2),
 ('الفرات', 2),
 ('عشتروت', 2),
 ('البحرين', 2),
 ('الخليج', 2),
 ('عسقلان', 2),
 ('سرجون', 2),
 ('الجوف', 2),
 ('نينوى', 2),
 ('مادي', 2),
 ('إيسوس', 2),
 ('الهند', 2),
 ('تغران', 2),
 ('الرومان', 2),
 ('سليح', 2),
 ('الأندلس', 2),
 ('المدينة', 2),
 ('الموصل', 2),
 ('بغداد', 2),
 ('أميريكة', 1),
 ('شام', 1),
 ('آشورية', 1),
 ('طوروس', 1),
 ('ج

In [ ]:
#generate_output_csv("/content/drive/MyDrive/Dr._Zaki_Assistance/ARA Project/countries - Sheet1.csv" )

KeyError: ignored

In [ ]:
#convert CSV to tuple
filtered_locations = csv_to_tuples("/content/drive/MyDrive/Dr._Zaki_Assistance/ARA Project/location-freq-filtered-Sheet1.csv")

In [ ]:
locations_freq_with_desc = add_column_to_tuples(locations_freq, "/content/drive/MyDrive/Dr._Zaki_Assistance/ARA Project/countries - Sheet1.csv")

In [ ]:
locations_freq_with_desc

[('سورية', 40),
 ('دمشق', 28),
 ('الشام', 27),
 ('مصر', 17),
 ('بابل', 13),
 ('العراق', 10),
 ('غسان', 9),
 ('يهوذا', 9),
 ('آشور', 9),
 ('فلسطين', 8),
 ('الأنباط', 7),
 ('كنعان', 6),
 ('إسرائيل', 6),
 ('اليمن', 6),
 ('اليونان', 6),
 ('رومة', 6),
 ('صور', 5),
 ('البتراء', 5),
 ('الروم', 4),
 ('فينيقية', 4),
 ('تدمر', 4),
 ('فارس', 4),
 ('تنوخ', 4),
 ('بالشام', 4),
 ('لبنان', 3),
 ('البحر', 3),
 ('قرقميش', 3),
 ('أورشليم', 3),
 ('دومة', 3),
 ('أفريقية', 3),
 ('الأردن', 3),
 ('أنطاكية', 3),
 ('حمص', 3),
 ('بدمشق', 3),
 ('القسطنطينية', 3),
 ('أوروبة', 3),
 ('الحجاز', 3),
 ('حلب', 3),
 ('بلبنان', 2),
 ('المملكة', 2),
 ('العقبة', 2),
 ('الحثيين', 2),
 ('الفرات', 2),
 ('عشتروت', 2),
 ('البحرين', 2),
 ('الخليج', 2),
 ('عسقلان', 2),
 ('سرجون', 2),
 ('الجوف', 2),
 ('نينوى', 2),
 ('مادي', 2),
 ('إيسوس', 2),
 ('الهند', 2),
 ('تغران', 2),
 ('الرومان', 2),
 ('سليح', 2),
 ('الأندلس', 2),
 ('المدينة', 2),
 ('الموصل', 2),
 ('بغداد', 2),
 ('أميريكة', 1),
 ('شام', 1),
 ('آشورية', 1),
 ('طوروس', 1),
 ('ج

In [ ]:
#This line takes the location names from the tuple
locations_only = [t[0] for t in locations_freq]

In [ ]:
locations_only

['سورية',
 'دمشق',
 'الشام',
 'مصر',
 'بابل',
 'العراق',
 'غسان',
 'يهوذا',
 'آشور',
 'فلسطين',
 'الأنباط',
 'كنعان',
 'إسرائيل',
 'اليمن',
 'اليونان',
 'رومة',
 'صور',
 'البتراء',
 'الروم',
 'فينيقية',
 'تدمر',
 'فارس',
 'تنوخ',
 'بالشام',
 'لبنان',
 'البحر',
 'قرقميش',
 'أورشليم',
 'دومة',
 'أفريقية',
 'الأردن',
 'أنطاكية',
 'حمص',
 'بدمشق',
 'القسطنطينية',
 'أوروبة',
 'الحجاز',
 'حلب',
 'بلبنان',
 'المملكة',
 'العقبة',
 'الحثيين',
 'الفرات',
 'عشتروت',
 'البحرين',
 'الخليج',
 'عسقلان',
 'سرجون',
 'الجوف',
 'نينوى',
 'مادي',
 'إيسوس',
 'الهند',
 'تغران',
 'الرومان',
 'سليح',
 'الأندلس',
 'المدينة',
 'الموصل',
 'بغداد',
 'أميريكة',
 'شام',
 'آشورية',
 'طوروس',
 'جرابلس',
 'طرطوس',
 'حام',
 'يشوع',
 'بني',
 'شكيم',
 'نابلس',
 'خراد',
 'آرام',
 'كريت',
 'غزة',
 'لمصر',
 'النيل',
 'الكلب',
 'الكلدانيين',
 'بنيويورك',
 'عنزى',
 'السامرة',
 'تلحونة',
 'بنوبال',
 'عسير',
 'قبرص',
 'ظافرة',
 'آسية',
 'الأناضول',
 'خليج',
 'جبيل',
 'دارا',
 'صيدا',
 'أرمينية',
 'بالشرق',
 'الأدوميين',
 'فالبت

In [ ]:
marker_locations = markers_arabic_places(filtered_locations, "686e5750dedb47ada75ceca950e88378")
locations_map = draw_arrows_to_main_marker(marker_locations)

In [ ]:
locations_map.save('/content/drive/MyDrive/Dr._Zaki_Assistance/ARA Project/map.html')